In [161]:
import pandas as pd
import numpy as np
import sklearn
from collections import Counter
from itertools import dropwhile
from sklearn.cluster import KMeans
import matplotlib.pyplot

print("Import Complete")


Import Complete


In [162]:
def build_vocabulary(messages, word_drop=True):
        vocabulary = Counter()
        print(len(messages))
        for message in messages:
            message = str(message)
            #message = remove_nonalphanumeric(message)
            message_split = message.split()
            gram_count = 1
            grams = []
            for i in range(len(message_split) - 1):
                gram = ''
                for n in range(gram_count):
                    gram = gram + message_split[i+n] + ' '
                grams.append(gram[:-1])
            vocabulary = vocabulary + Counter(message_split) + Counter(grams)
        print(len(vocabulary))
        if word_drop == True:
            for key, count in dropwhile(
                                        lambda key_count: key_count[1] >= (len(messages) * .01),
                                        vocabulary.most_common()):
                del vocabulary[key]

        return vocabulary

def remove_nonalphanumeric(message):
    #try:        
    message = message.lower()
    delchar_table = {ord(c): None for c in message if c not in 'abcdefghijklmnopqrstuvwxyz0123456789 '}
    print(type(message))
    return message.translate(None, delchar_table)

def label_features(df, features_master):
        for i,row in df.iterrows():
            #message = pruning_dict.remove_nonalphanumeric(row.text)
            message = str(row.question_class)
            features = Counter(message.split()) & features_master
            features = features + features_master
            features = list(np.array(list(features.values())) - 1)
            df.set_value(i,'features',features)
        return df
    
def create_feature_dataframe(df, features_master):
    return pd.DataFrame(list(df.features), columns=range(len(features_master)))

def cluster_filter2(df, df2, N, features_master):
    print("Start loop")
    singles = 0
    clusterer = KMeans(n_clusters=N)
    clusterer.fit(df2)
    transform = clusterer.transform(df2)
    d_center = []
    cluster = []
    for x in transform:
        d_center.append(min(x)**2)
        cluster.append(np.argmin(x))
    df['cluster'] = cluster
    df['d_from_center'] = d_center
    d_center = np.array(d_center)
    mean = np.mean(d_center)
    std = np.std(d_center)
    for cgroup in range(N):
        group = df.groupby('cluster').get_group(cgroup)
        if group.question_class.count() <= 5:
            df = df.drop(group.index)
            singles += 1
    print("# of singles: {}".format(singles))
    print("df length: {}".format(len(df)))
    if singles >= 6 and N <= (len(df) - singles):
        df = cluster_filter2(df, create_feature_dataframe(df, features_master), N, features_master)
        return df
    for cgroup in range(N):
        try:
            gscore = 0.0
            group = df.groupby('cluster').get_group(cgroup)
            for i, row in group.iterrows():
                z = (row.d_from_center - mean) / std
                if z < -0.68:
                    gscore += 1
            glength = len(group)
            gpercent = gscore/glength

            print("Found {} messages of the same form with a gpercent of {}.".format(len(group), gpercent))
            for message in group.question_class.head(3):
                if group.question_class.count() > 1:
                    print(message)
                    print("")
            print("")
        except: pass
    
    
    return df
    
    
def cluster_filter(df, df2, N):
    clusterer = KMeans(n_clusters=N)
    clusterer.fit(df2)
    transform = clusterer.transform(df2)
    d_center = []
    cluster = []
    for x in transform:
        d_center.append(min(x)**2)
        cluster.append(np.argmin(x))
    df['d_from_center'] = d_center
    df['cluster'] = cluster
    d_center = np.array(d_center)
    mean = np.mean(d_center)
    std = np.std(d_center)
    for cgroup in range(N):
        gscore = 0
        group = df.groupby('cluster').get_group(cgroup)
        for i, row in group.iterrows():
            z = (row.d_from_center - mean) / std
            if z < -0.68:
                gscore += 1
        glength = len(group)
        gpercent = gscore/glength
        print("Found {} messages of the same form.".format(len(group)))
        for message in group.question_class.head(3):
            if group.question_class.count() > 1:
                print(message)
                print("")
        print("")
        '''
        if glength > 1 and gpercent > .9:
            print("Identified the following message as SPAM.")
            print("Found {} messages of the same form.".format(len(group)))
            for message in group.text.head(1):
                print(message)
            print("")
            df = df.drop(group.index)  
        '''
    return df


In [163]:
csvdata = "./data/subcategory_2_think_hr_v2.csv"
df = pd.read_csv(csvdata)
df = df[['question_class']][:5000]
vocab = build_vocabulary(df.question_class)
print(len(vocab))
features_master = Counter(list(vocab.keys()))
df["features"] = [[0] * len(vocab)] * len(df)
df = label_features(df, features_master)
df2 = create_feature_dataframe(df, features_master)

df = cluster_filter2(df, df2, 15, features_master)





5000
8757
1015


C:\Users\kerri\Anaconda2\envs\thinkhr\lib\site-packages\ipykernel_launcher.py:39: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Start loop
# of singles: 6
df length: 4990
Start loop
# of singles: 4
df length: 4985
Found 1491 messages of the same form with a gpercent of 0.361502347418.
via vm calling see vacation time sick time accrue people work overtime massachusetts company vermont location

question use intellectual property employment ends please give call

several questions following topics


Found 448 messages of the same form with a gpercent of 0.0647321428571.
work secure building badged video surveillance front going put sign front door employees need sign monitoring consent colorado information sample policies difference security staff regular staff regard policies

allow employees day personal loa job protected loa days offer cobra elect sign cobra pay reinstate coverage deny opportunity continue coverage lack payment

update client asking providing notice employees regarding ab provide information could locate information thinkhr site ab provide notice employees regarding rights victim domestic viol